# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

It contains the following 6 fields:

target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

ids: The id of the tweet ( 2087)

date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

flag: The query (lyx). If there is no query, then this value is NO_QUERY.

user: the user that tweeted (robotickilldozr)

text: the text of the tweet (Lyx is cool)



In [5]:
import numpy as np
import pandas as pd
import re 
import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

In [6]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hollydalton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hollydalton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hollydalton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
tweets = pd.read_csv('/Users/hollydalton/Desktop/twitter.csv', encoding='ISO-8859-1', header = None)
tweets.columns = ['target','id','date','flag','user','text']
tweet_samp = tweets.sample(20000)
tweet_samp

,target,id,date,flag,user,text
1093016,4,1970027062,Sat May 30 02:09:18 PDT 2009,NO_QUERY,mandylicious123,free gey marriage support stickers.! http://...
548464,0,2202374961,Tue Jun 16 21:41:21 PDT 2009,NO_QUERY,giegiebaby,i want my phoneee back. haha
676713,0,2248476996,Fri Jun 19 20:41:12 PDT 2009,NO_QUERY,kservss,Glad i have to drive in this
1182142,4,1982200335,Sun May 31 10:43:48 PDT 2009,NO_QUERY,kpizza408,@DwightHoward you guys played some amazing gam...
1431041,4,2060015517,Sat Jun 06 18:05:33 PDT 2009,NO_QUERY,tlopezmusic,Singing through a chest cold tonight in Limbo ...
...,...,...,...,...,...,...
795862,0,2327529437,Thu Jun 25 08:27:48 PDT 2009,NO_QUERY,MrMiggz,Getting ready for work. Where's wifey
1079822,4,1968135020,Fri May 29 20:49:35 PDT 2009,NO_QUERY,MissSimone32,Heyy @Cash550
1316257,4,2014135225,Wed Jun 03 00:33:51 PDT 2009,NO_QUERY,abbymacabia,playing pet society on facebook..
1432170,4,2060209822,Sat Jun 06 18:28:16 PDT 2009,NO_QUERY,Bumblefuck,"@InorganicBoy Cool, must get it Hopefully the..."


In [11]:
tweet_samp['target'].value_counts()   #interesting to see there are no neutral (2) target texts

4    10095
0     9905
Name: target, dtype: int64

In [12]:
tweet_samp['flag'].value_counts()  #only one value in flag too 

NO_QUERY    20000
Name: flag, dtype: int64

In [13]:
tweet_samp['user'].value_counts() 

shanajaca        7
tweetpet         7
lost_dog         7
DarkPiano        6
MissXu           6
                ..
AvonteNikole     1
PlasticFunnel    1
bepositivebud    1
simplowdfan      1
tigermel         1
Name: user, Length: 18867, dtype: int64

In [14]:
tweet_samp.isnull().sum().sum()  #no nulls


0

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

### First we define the functions (again, thanks to Asha!!)

In [15]:
def clean_up(c):
    
    lower = c.lower()
    no_url = re.sub('http\S+', '',lower)
    string_list = re.findall('[a-z]+',no_url)

    return ' '.join(string_list)

In [16]:
def tokenize(t):
    return nltk.word_tokenize(t)

In [17]:
def stem_and_lemmatize(l):
    
    ls_list = [] 
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    for word in l:
        lem = lemmatizer.lemmatize(word)
        stem = stemmer.stem(lem)
        ls_list.append(stem)
    
    return ls_list

In [18]:
def remove_stopwords(l):
    
    stop_words = set(stopwords.words('english'))
    
    filtered_sentence = [w for w in l if not w in stop_words]
    
    return filtered_sentence

In [19]:
#then apply the functions to the tweet_samp dataframe

tweet_samp['text_processed'] = tweet_samp['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

tweet_samp

,target,id,date,flag,user,text,text_processed
1093016,4,1970027062,Sat May 30 02:09:18 PDT 2009,NO_QUERY,mandylicious123,free gey marriage support stickers.! http://...,"[free, gey, marriag, support, sticker]"
548464,0,2202374961,Tue Jun 16 21:41:21 PDT 2009,NO_QUERY,giegiebaby,i want my phoneee back. haha,"[want, phonee, back, haha]"
676713,0,2248476996,Fri Jun 19 20:41:12 PDT 2009,NO_QUERY,kservss,Glad i have to drive in this,"[glad, drive, thi]"
1182142,4,1982200335,Sun May 31 10:43:48 PDT 2009,NO_QUERY,kpizza408,@DwightHoward you guys played some amazing gam...,"[dwighthoward, guy, play, amaz, game, earn, th..."
1431041,4,2060015517,Sat Jun 06 18:05:33 PDT 2009,NO_QUERY,tlopezmusic,Singing through a chest cold tonight in Limbo ...,"[sing, chest, cold, tonight, limbo, music]"
...,...,...,...,...,...,...,...
795862,0,2327529437,Thu Jun 25 08:27:48 PDT 2009,NO_QUERY,MrMiggz,Getting ready for work. Where's wifey,"[get, readi, work, wifey]"
1079822,4,1968135020,Fri May 29 20:49:35 PDT 2009,NO_QUERY,MissSimone32,Heyy @Cash550,"[heyi, cash]"
1316257,4,2014135225,Wed Jun 03 00:33:51 PDT 2009,NO_QUERY,abbymacabia,playing pet society on facebook..,"[play, pet, societi, facebook]"
1432170,4,2060209822,Sat Jun 06 18:28:16 PDT 2009,NO_QUERY,Bumblefuck,"@InorganicBoy Cool, must get it Hopefully the...","[inorganicboy, cool, must, get, hope, come, au..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [21]:
freq_dist = nltk.FreqDist()    #library from NLTK to calculate frequency distribution
      

In [23]:
for each_tweet in tweet_samp['text_processed']:     #use this function to add in a tweet tokenizer
    for each_word in each_tweet:
        freq_dist[each_word] += 1

#https://stackoverflow.com/questions/4634787/freqdist-with-nltk


In [24]:
bag_of_words = list(freq_dist.items())[:5000]
bag_of_words

[('free', 101),
 ('gey', 1),
 ('marriag', 6),
 ('support', 39),
 ('sticker', 2),
 ('want', 704),
 ('phonee', 2),
 ('back', 759),
 ('haha', 412),
 ('glad', 109),
 ('drive', 95),
 ('thi', 1147),
 ('dwighthoward', 1),
 ('guy', 267),
 ('play', 242),
 ('amaz', 162),
 ('game', 161),
 ('earn', 5),
 ('right', 360),
 ('let', 263),
 ('kick', 34),
 ('la', 71),
 ('sing', 42),
 ('chest', 9),
 ('cold', 99),
 ('tonight', 301),
 ('limbo', 2),
 ('music', 142),
 ('modestcowboydan', 1),
 ('watch', 554),
 ('via', 50),
 ('www', 65),
 ('tdpodcast', 1),
 ('com', 110),
 ('work', 1102),
 ('boo', 76),
 ('dakgirl', 1),
 ('still', 508),
 ('bloom', 2),
 ('could', 264),
 ('sworn', 2),
 ('jazgar', 1),
 ('wa', 1300),
 ('ak', 2),
 ('sorri', 334),
 ('great', 394),
 ('though', 325),
 ('hahaha', 104),
 ('ye', 257),
 ('realli', 634),
 ('ive', 37),
 ('count', 28),
 ('dari', 3),
 ('dulu', 1),
 ('dah', 3),
 ('know', 722),
 ('year', 204),
 ('may', 104),
 ('special', 44),
 ('ahh', 38),
 ('upgrad', 15),
 ('bet', 40),
 ('come', 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [1]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here